# Chapter 4. Basic Quantum Theory

**Programming Drill 4.1.1** Write a program that simulates the first quantum system described in this section. The user should be able to specify how many points the particle can occupy (warning: keep the max number low, or you will fairly quickly run out of memory). The user will also specify a ket state vector by assigning its amplitudes.

The program, when asked the likelihood of finding the particle at a given point, will perform the calculations described in Example 4.1.1. If the user enters two kets, the system will calculate the probability of transitioning from the first ket to the second, after an observation has been made.

In [29]:
def ket_to_posibility(ket: list[complex]):
  norm_squareds = [c.real**2 + c.imag**2 for c in ket]
  total = sum(norm_squareds)
  return [ns/total for ns in norm_squareds]

print("The possibilities that the particle would be at each entry: ", ket_to_posibility([-3-1j, -2j, 1j, 2]))

def conjugate_transpose_matrix(C: list[list[complex]]):
  res = []
  for i in range(len(C[0])):
    res.append([])
    for j in range(len(C)):
      res[i].append(C[j][i].conjugate())
  return res

def multiply_matrixes(A: list[list[complex]], B: list[list[complex]]):
  ma = len(A)
  na = len(A[0])
  mb = len(B)
  nb = len(B[0])
  if na != mb:
    raise Exception("Inappropriate size!")
  res = []
  for i in range(ma):
    res.append([])
    for j in range(nb):
      sum = 0
      for k in range(na):
        sum += A[i][k]*B[k][j]
      res[i].append(sum)
  return res

def to_column_matrix(row_vector):
  return [[row] for row in row_vector]

import math

def calc_vector_norm(vector: list[int]):
  return math.sqrt(sum(c.real**2 + c.imag**2 for c in vector))

def calc_transition_amplitude(source, target):
  bra = conjugate_transpose_matrix(to_column_matrix(target))
  ket = to_column_matrix(source)
  denominator = calc_vector_norm(source)*calc_vector_norm(target)
  bracket = multiply_matrixes(bra, ket)[0][0]
  amplitude: complex = bracket/denominator
  posibility = math.sqrt(amplitude.real**2 + amplitude.imag**2)
  return bracket/denominator, round(posibility)

print("The transition amplitude and its corresponding probability: ", calc_transition_amplitude(
  [1, -1j],
  [1j, 1]
))

The possibilities that the particle would be at each entry:  [0.5263157894736842, 0.21052631578947367, 0.05263157894736842, 0.21052631578947367]
The transition amplitude and its corresponding probability:  (-0.9999999999999998j, 1)


**Programming Drill 4.2.1** Continue your simulation of a quantum system by adding observables to the picture: the user will input a square matrix of the appropriate size, and a ket vector. The program will verify that the matrix is hermitian, and if so, it will calculate the mean value and the variance of the observable on the given state.

In [30]:
def is_hermitian(matrix: list[list[complex]]):
  for i in range(len(matrix)):
    for j in range(len(matrix[i])):
      if matrix[i][j] != matrix[j][i].conjugate():
        return False
  return True

print(is_hermitian([
  [-1, -1j],
  [1j, 1]
])) # Example 4.2.1

def apply_observable(ohm, psi):
  result = multiply_matrixes(ohm, to_column_matrix(psi))
  return result
  
print(apply_observable([
  [-1, -1j],
  [1j, 1]
], [-1, -1-1j])) # Example 4.2.1

def observe_mean(ohm, psi):
  target = apply_observable(ohm, psi)
  bra = conjugate_transpose_matrix(target)
  ket = to_column_matrix(psi)
  mean = multiply_matrixes(bra, ket)[0][0]
  return mean

print(observe_mean([
  [1, -1j],
  [1j, 2]
], [2**0.5/2, 2**0.5/2*1j])) # Example 4.2.5

print(observe_mean([
  [3, 1+2j],
  [1-2j, -1]
], [2**0.5/2, -2**0.5/2])) # Exercise 4.2.9

def observe_variance(ohm, psi):
  mean = observe_mean(ohm, psi)
  mean_identity_matrix = []
  for i in range(len(ohm)):
    mean_identity_matrix.append([])
    for j in range(len(ohm[i])):
      if i == j:
        mean_identity_matrix[i].append(mean)
      else:
        mean_identity_matrix[i].append(0)

  delta_ohm = []
  for i in range(len(ohm)):
    delta_ohm.append([])
    for j in range(len(ohm[i])):
      delta_ohm[i].append(ohm[i][j] - mean_identity_matrix[i][j])
  
  print("> What then is the mean of Δψ(Ω) itself on the normalized state |ψ⟩? A simple calculation shows that it is precisely zero: ", observe_mean(delta_ohm, psi)) # Exercise 4.2.10

  delta_ohm_squared = multiply_matrixes(delta_ohm, delta_ohm)

  return observe_mean(delta_ohm_squared, psi)

print(observe_variance([
  [1, -1j],
  [1j, 2]
], [2**0.5/2, 2**0.5/2*1j])) # Example 4.2.7


True
[[1j], [(-1-2j)]]
(2.5000000000000004+0j)
0j
> What then is the mean of Δψ(Ω) itself on the normalized state |ψ⟩? A simple calculation shows that it is precisely zero:  (-4.163336342344337e-16+0j)
(0.25+0j)


**Programming Drill 4.3.1** Next step in the simulation: when the user enters an observable and a state vector, the program will return the list of eigenvalues of the observable, the mean value of the observable on the state, and the probability that the state will transition to each one of the eigenstates. Optional: plot the corresponding probability distribution.

In [31]:
import numpy as np

def norm(c: complex):
  return (c.real**2 + c.imag**2)**0.5

def measure(observable: list[list[complex]], state: list[complex]):
  eigenvalues, eigenvectors = np.linalg.eig(np.array(observable, dtype=complex))
  print("Eigenvalues of the observable:", eigenvalues)
  mean = observe_mean(observable, state)
  print("Mean value of the observable on the state:", mean)
  probabilities = [norm(calc_transition_amplitude(state, ev)[0])**2 for ev in eigenvectors]
  print("Probability that the state will transition to each one of the eigenstates:", probabilities)

measure([
  [-1, -1j],
  [1j, 1]
], [1/2, 1/2])

measure([
  [-1, -1j],
  [1j, 1]
], [0, 1])

Eigenvalues of the observable: [-1.41421356+0.j  1.41421356+0.j]
Mean value of the observable on the state: 0j
Probability that the state will transition to each one of the eigenstates: [0.4999999999999999, 0.4999999999999999]
Eigenvalues of the observable: [-1.41421356+0.j  1.41421356+0.j]
Mean value of the observable on the state: (1+0j)
Probability that the state will transition to each one of the eigenstates: [0.1464466094067262, 0.8535533905932737]


**Programming Drill 4.4.1** Add dynamics to your computer simulation of the particle on a grid: the user should input a number of time steps n, and a corresponding sequence of unitary matrices U<sub>n</sub> of the appropriate size. The program will then compute the state vector after the entire sequence U<sub>n</sub> has been applied.

In [32]:
def apply_dynamic(state: list[complex], dynamic: list[list[complex]], steps: int, show_posibility=False, rewind=False):
  current_state = to_column_matrix(state)
  for step in range(steps):
    current_state = multiply_matrixes(dynamic, current_state)
    print(f"Step {step + 1}:", current_state)
    if show_posibility:
      print(f"Posibilities:", ket_to_posibility([row[0] for row in current_state]))

  if rewind:
    inverse_matrix = conjugate_transpose_matrix(dynamic)
    for step in range(steps):
      current_state = multiply_matrixes(inverse_matrix, current_state)
      print(f"Step -{steps - step}:", current_state)
      if show_posibility:
        print(f"Posibilities:", ket_to_posibility([row[0] for row in current_state]))
  return current_state

print("Final state:", apply_dynamic(
  [1, 0, 0, 0],
  [
    [0, 1/2**0.5, 1/2**0.5, 0],
    [1j*1/2**0.5, 0, 0, 1/2**0.5],
    [1/2**0.5, 0, 0, 1j*1/2**0.5],
    [0, 1/2**0.5, -1/2**0.5, 0],
  ],
  3,
  show_posibility=True
)) # Exercise 4.4.2

print("Final state: ", apply_dynamic(
  [1, 0, 0, 0],
  [
    [0, 1/2**0.5, 1/2**0.5, 0],
    [1j*1/2**0.5, 0, 0, 1/2**0.5],
    [1/2**0.5, 0, 0, 1j*1/2**0.5],
    [0, 1/2**0.5, -1/2**0.5, 0],
  ],
  3,
  rewind=True
))

Step 1: [[0.0], [0.7071067811865475j], [(0.7071067811865475+0j)], [0.0]]
Posibilities: [0.0, 0.5, 0.5, 0.0]
Step 2: [[(0.4999999999999999+0.4999999999999999j)], [0j], [0j], [(-0.4999999999999999+0.4999999999999999j)]]
Posibilities: [0.5, 0.0, 0.0, 0.5]
Step 3: [[0j], [(-0.7071067811865474+0.7071067811865474j)], [0j], [0j]]
Posibilities: [0.0, 1.0, 0.0, 0.0]
Final state: [[0j], [(-0.7071067811865474+0.7071067811865474j)], [0j], [0j]]
Step 1: [[0.0], [0.7071067811865475j], [(0.7071067811865475+0j)], [0.0]]
Step 2: [[(0.4999999999999999+0.4999999999999999j)], [0j], [0j], [(-0.4999999999999999+0.4999999999999999j)]]
Step 3: [[0j], [(-0.7071067811865474+0.7071067811865474j)], [0j], [0j]]
Step -3: [[(0.49999999999999983+0.49999999999999983j)], [0j], [0j], [(-0.49999999999999983+0.49999999999999983j)]]
Step -2: [[0j], [0.7071067811865472j], [(0.7071067811865472+0j)], [0j]]
Step -1: [[(0.9999999999999996+0j)], [0j], [0j], [0j]]
Final state:  [[(0.9999999999999996+0j)], [0j], [0j], [0j]]


**Programming Drill 4.5.1** Expand the simulation of the last sections by letting the user choose the number of particles.

In [33]:
from scipy.stats import unitary_group

def assemble_system(particles: list[int], positions: int):
  state = []
  for position in particles:
    for i in range(positions):
      if position == i:
        state.append(1)
      else:
        state.append(0)
  
  dynamic = unitary_group.rvs(len(particles)*positions)
  print("Initial state:", state)
  print("Generated dynamic:", dynamic)
  return state, dynamic

print("Final state:", apply_dynamic(
  *assemble_system(particles=[2, 0, 1], positions=4),
  steps=3,
  show_posibility=True,
))

Initial state: [0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0]
Generated dynamic: [[ 0.05104136+5.93251295e-02j -0.1898574 -1.58700794e-02j
  -0.03482025+1.40671017e-01j  0.13488897-3.43058675e-02j
  -0.0043302 +1.91267114e-01j -0.01012022-3.49922855e-02j
  -0.54701922-1.27586971e-01j -0.17539401-4.51554401e-01j
  -0.15936328-2.60944774e-03j  0.33423978+3.30105697e-01j
   0.08577054+1.50183394e-01j  0.10096313-2.07143947e-01j]
 [-0.15875709-1.60612009e-01j -0.28392349+1.14188729e-01j
  -0.0398491 -8.65391636e-02j  0.01137324+1.78424647e-01j
   0.43416728+2.97266563e-01j  0.02014133-3.87562824e-02j
  -0.07587321-6.72432357e-03j  0.19165997-2.20310405e-01j
  -0.19309135+7.06253552e-05j -0.10731051-1.81662366e-01j
  -0.32428172+2.42744471e-02j -0.4484331 +2.36256164e-01j]
 [ 0.27657321-1.65658728e-01j  0.08871529-7.36556422e-02j
   0.28516289+4.62877289e-02j  0.14038387+1.34572529e-01j
   0.04549422-2.44222736e-02j -0.12021676+2.15443196e-01j
   0.39188823-2.00488089e-01j -0.13342398+4.06976713e-02j